In [175]:
import pandas as pd
import os
from glob import glob
from functools import partial

from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, TaskType, AdaptionPromptConfig, get_peft_model
from datasets import Dataset
from transformers import TrainingArguments, EarlyStoppingCallback

from torch.utils.data import DataLoader

import numpy as np

import evaluate

# Model choice

In [19]:
MODEL_CHOICE = 2
if MODEL_CHOICE == 1:
    model_name = "facebook/opt-350m"
    model = AutoModel.from_pretrained(model_name)
elif MODEL_CHOICE == 2:
    model_name = "google-bert/bert-base-cased"
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

model_dir = model_name.replace("/", "-")

/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 591,362 || all params: 108,903,172 || trainable%: 0.5430163227936097


# load data

In [103]:
df = pd.concat(
    [
        pd.DataFrame(data={"text": [open(f).readline() for f in glob("/Users/francois.weber/datasets/aclImdb/train/neg/*.txt")]}).assign(label=0),
        pd.DataFrame(data={"text": [open(f).readline() for f in glob("/Users/francois.weber/datasets/aclImdb/train/pos/*.txt")]}).assign(label=1),
    ]
).sample(frac=1.0).reset_index(drop=True)

In [214]:
df.sample(3)

,text,label
6868,Many reviews I've read reveals that most peopl...,1
24016,Former brat pack actor and all round pretty bo...,0
9668,Once upon a time Hollywood produced live-actio...,1


# Pick a model

In [124]:
MODEL_NAME = "google-bert/bert-base-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Tools

In [126]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [151]:
metrics = {
    "precision": evaluate.load("precision"),
    "recall": evaluate.load("recall"),
    "f1": evaluate.load("f1"),
}

In [164]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {name: metric.compute(predictions=predictions, references=labels) for name, metric in metrics.items()}

# Prepare dataset

In [ ]:
dataset = Dataset.from_pandas(df).train_test_split(0.8)

In [127]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████████████████████████████████████████████████████████████████| 20000/20000 [00:02<00:00, 7634.07 examples/s]


In [128]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

# Models fine-tuning !

In [176]:
earlystop_cb = EarlyStoppingCallback(early_stopping_patience=3)

## Baseline with a default FT

In [181]:
training_args = TrainingArguments(
    output_dir=MODEL_NAME.replace("/", "-") + "-RAW",
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3, 
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [182]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.465300,0.337309,{'accuracy': 0.86285},{'precision': 0.8616798162388895},{'recall': 0.8640096134588424},{'f1': 0.8628431421571079}
2,0.405600,0.381923,{'accuracy': 0.86515},{'precision': 0.8617369727047146},{'recall': 0.8694171840576808},{'f1': 0.8655600418722895}
3,0.379500,0.450804,{'accuracy': 0.88085},{'precision': 0.889458047331216},{'recall': 0.8694171840576808},{'f1': 0.8793234415354231}
4,0.277600,0.460298,{'accuracy': 0.88705},{'precision': 0.9077572559366754},{'recall': 0.8613058281594232},{'f1': 0.883921689532912}


TrainOutput(global_step=2500, training_loss=0.3722097229003906, metrics={'train_runtime': 5860.0452, 'train_samples_per_second': 4.266, 'train_steps_per_second': 0.533, 'total_flos': 5266845450240000.0, 'train_loss': 0.3722097229003906, 'epoch': 4.0})

In [190]:
model.save_pretrained("./raw")

## LoRA

In [197]:
FT_TYPE = "LORA"

### Tiny LoRA r=2

In [198]:
r = 2

In [199]:
lora_config = LoraConfig(
    r=r,
    target_modules=["query", "value"],
    task_type=TaskType.SEQ_CLS,
    lora_alpha=32,
    lora_dropout=0.05
)

In [200]:
lora_model = get_peft_model(AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2), lora_config)
lora_model.print_trainable_parameters()

/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 75,266 || all params: 108,387,076 || trainable%: 0.0694418585477848


In [201]:
training_args = TrainingArguments(
    output_dir=MODEL_NAME.replace("/", "-") + "-" + FT_TYPE,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3, 
    load_best_model_at_end=True
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [202]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.437700,0.320221,{'accuracy': 0.8665},{'precision': 0.8905616058082426},{'recall': 0.8352693771279792},{'f1': 0.8620297643654402}
2,0.362100,0.330836,{'accuracy': 0.88155},{'precision': 0.8979208024239891},{'recall': 0.8606048467854996},{'f1': 0.8788669018765659}
3,0.356200,0.407664,{'accuracy': 0.8848},{'precision': 0.9052542730533868},{'recall': 0.8592028840376527},{'f1': 0.8816276202219482}
4,0.263900,0.468798,{'accuracy': 0.8924},{'precision': 0.9038977108682202},{'recall': 0.8778289605447627},{'f1': 0.8906726275147328}


/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want t

TrainOutput(global_step=2500, training_loss=0.3470293090820312, metrics={'train_runtime': 5727.1155, 'train_samples_per_second': 4.365, 'train_steps_per_second': 0.546, 'total_flos': 5266845450240000.0, 'train_loss': 0.3470293090820312, 'epoch': 4.0})

In [203]:
lora_model.save_pretrained(f"./lora-r={r}")

/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Regular LoRA r=16

In [208]:
r = 16

In [209]:
lora_config = LoraConfig(
    r=r,
    target_modules=["query", "value"],
    task_type=TaskType.SEQ_CLS,
    lora_alpha=32,
    lora_dropout=0.05
)

In [210]:
lora_model = get_peft_model(AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2), lora_config)
lora_model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 591,362 || all params: 108,903,172 || trainable%: 0.5430163227936097


In [211]:
training_args = TrainingArguments(
    output_dir=MODEL_NAME.replace("/", "-") + "-" + FT_TYPE,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3, 
    load_best_model_at_end=True
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [212]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.465300,0.277267,{'accuracy': 0.88945},{'precision': 0.8895680929952902},{'recall': 0.8889445223312638},{'f1': 0.8892561983471075}
2,0.332300,0.359461,{'accuracy': 0.88155},{'precision': 0.8446918273146891},{'recall': 0.9346084518325656},{'f1': 0.887378179225101}
3,0.281900,0.359880,{'accuracy': 0.8828},{'precision': 0.9313614811469858},{'recall': 0.8261566192669738},{'f1': 0.8756102738272129}
4,0.154200,0.476943,{'accuracy': 0.8938},{'precision': 0.917569577225409},{'recall': 0.8650110154215902},{'f1': 0.8905154639175258}


/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want t

TrainOutput(global_step=2500, training_loss=0.2889537841796875, metrics={'train_runtime': 5881.2423, 'train_samples_per_second': 4.251, 'train_steps_per_second': 0.531, 'total_flos': 5298554388480000.0, 'train_loss': 0.2889537841796875, 'epoch': 4.0})

In [213]:
lora_model.save_pretrained(f"./lora-r={r}")

/Users/francois.weber/Library/Caches/pypoetry/virtualenvs/try-peft-yE8UJEJ6-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Prompt Tuning

In [167]:
FT_TYPE = "LORA"